In [1]:
import pandas as pd
import numpy as np
from demoparser2 import DemoParser
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Création d'un dictionnaire avec tous les df par matchs

In [4]:
import os
import pandas as pd
# Créer une liste pour stocker les DataFrames concaténés pour chaque fichier
all_matches = []

# Répertoire contenant les fichiers de démonstration
demo_directory = "D:/Python/projet_cs_v2/demos"

team = 'zobrux'

# Parcours des fichiers dans le répertoire
for filename in os.listdir(demo_directory):
    if filename.endswith(".dem"): 
        # Chargement de la démo
        demo = DemoParser(os.path.join(demo_directory, filename))

        # Extraction du tick max de la partie
        max_tick = demo.parse_event("round_end")["tick"].max()

        # Extraction du premier tick de la partie
        first_round_tick = demo.parse_event('round_start').drop([0,1,2]).loc[3, 'tick'] # on drop le knife round, le warmup round et le RR

        # Extraction de la carte 
        map = pd.DataFrame([demo.parse_header()])

        #tick fin période d'achat
        freezetime_end_tick = demo.parse_event("round_freeze_end")["tick"].drop([0,1]).tolist()# on drop le knife round et le warmup, pas de freezetime au RR

        
        # Extraction des noms des joueurs et des équipes 
        team_V3 = demo.parse_ticks(["team_clan_name","team_name"])
        team_V3.drop(team_V3[team_V3['tick'] < first_round_tick].index, inplace=True)
        team_V3.drop(team_V3[team_V3['tick'] > max_tick].index, inplace=True)
        team_V3.rename(columns={'team_name':'side'}, inplace=True)
        name_to_team = dict(zip(team_V3['name'], team_V3['team_clan_name']))

        #eco 
        eco = demo.parse_ticks(["current_equip_value", "total_rounds_played"], ticks=freezetime_end_tick)
        if eco['total_rounds_played'].min() == 0:
            eco['total_rounds_played'] = eco['total_rounds_played'] + 1
        eco['current_equip_value'] = eco['current_equip_value'] - 200
        #eco par équipe
        eco_by_players = pd.merge(eco,team_V3, on=['tick','name','steamid'])
        eco_by_team = eco_by_players.groupby(['total_rounds_played','team_clan_name','tick','side'])['current_equip_value'].sum().reset_index()

        # Kills DataFrame
        trade = demo.parse_event("player_death", other=["game_time", "round_start_time",'total_rounds_played'])
        trade['total_rounds_played'] = trade['total_rounds_played'] + 1
        trade.drop(trade[trade['tick'] < first_round_tick].index, inplace=True)
        trade["player_died_time"] = trade["game_time"] - trade["round_start_time"]
        trade = pd.merge(trade, team_V3[['team_clan_name','side','name','tick']], left_on=["attacker_name",'tick'], right_on=['name','tick'])
        trade = trade.rename(columns={'team_clan_name':'team_clan_name_attacker', 'side':'side_attacker'})
        trade = pd.merge(trade, team_V3[['team_clan_name','side','name','tick']], left_on=["user_name",'tick'], right_on=['name','tick'])
        trade = trade.rename(columns={'team_clan_name':'team_clan_name_user','side':'side_user'})

        # Damage DataFrame
        all_hits_regs = demo.parse_event("player_hurt", ticks=[max_tick])
        all_hits_regs.drop(all_hits_regs[all_hits_regs['tick'] < first_round_tick].index, inplace=True)
        all_hits_regs.drop(all_hits_regs[all_hits_regs['tick'] > max_tick].index, inplace=True)
        all_hits_regs['attacker_team'] = all_hits_regs['attacker_name'].map(name_to_team)
        all_hits_regs['user_team'] = all_hits_regs['user_name'].map(name_to_team)
        all_hits_regs = all_hits_regs[all_hits_regs['attacker_team'] != all_hits_regs['user_team']]

        # Stats DataFrame
        overall_stats = ["total_rounds_played","kills_total","assists_total","deaths_total", "mvps", "headshot_kills_total", 
                         "3k_rounds_total", "4k_rounds_total", "ace_rounds_total" ,"damage_total","utility_damage_total", 
                         "enemies_flashed_total","alive_time_total"]
        agg_stats = demo.parse_ticks(overall_stats, ticks=[max_tick])

        # round_winner
        round_ends = demo.parse_event("round_end", other=["total_rounds_played"])
        def transform_round_end(round_ends, first_round_tick):
            if round_ends.columns[0].lower() == 'legacy':
                round_ends['winner'] = round_ends['winner'].apply(lambda x: 'T' if x == 2 else 'CT' if x == 3 else 'unknown')
                def reason_mapping(x):
                    if x == 9:
                        return 'ct_killed'
                    elif x == 7:
                        return 'bomb_defused'
                    elif x == 8:
                        return 't_killed'
                    elif x == 12:
                        return 't_saved'
                    else:
                        return 'unknown'
                round_ends['reason'] = round_ends['reason'].apply(reason_mapping)
                round_ends.drop(round_ends[round_ends['tick'] <= first_round_tick].index, inplace=True)
                round_ends = round_ends[['reason', 'tick', 'total_rounds_played', 'winner']]
                round_ends['total_rounds_played'] = round_ends['total_rounds_played'] + 1
            else:
                round_ends.drop(round_ends[round_ends['tick'] <= first_round_tick].index, inplace=True)
                round_ends.drop(columns=['round'], inplace=True)
            return round_ends
        round_ends = transform_round_end(round_ends, first_round_tick)


        # Identification de l'équipe adverse
        unique_teams = team_V3['team_clan_name'].unique()
        adversary_team = [t for t in unique_teams if t != team][0]  # Trouver l'équipe adverse

        # Sélection des joueurs spécifiés dans zobrux
        df_team = team_V3.loc[team_V3['team_clan_name'] == team, :]

        # Ajouter une colonne avec l'équipe adverse comme Game_id
        df_team['Game_id'] = "VS_"+adversary_team

        # Stocker chaque DataFrame dans un dictionnaire pour ce fichier, y compris eco_info
        match_data = {
            'team_info': df_team,
            'trade_info': trade,
            'damage_info': all_hits_regs,
            'agg_stats': agg_stats,
            'eco_info': eco_by_team,
            'round_info': round_ends
        }

        # Ajout du dictionnaire à la liste
        all_matches.append(match_data)


C:\Users\Armel\AppData\Local\Temp\ipykernel_3220\1912450564.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  round_ends['total_rounds_played'] = round_ends['total_rounds_played'] + 1
C:\Users\Armel\AppData\Local\Temp\ipykernel_3220\1912450564.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_team['Game_id'] = "VS_"+adversary_team
C:\Users\Armel\AppData\Local\Temp\ipykernel_3220\1912450564.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

#### Analyse des df de chaque matchs :

all_matches[id du match][data souhaitée]

In [5]:
all_matches[1]['round_info']

KeyError: 'round_info'

#### Analyse multi matchs

In [6]:
joueurs = pd.DataFrame()
joueurs['name']  = team_V3.loc[(team_V3['team_clan_name'] == 'zobrux') & (team_V3['tick'] == first_round_tick), 'name']
joueurs = joueurs.reset_index().drop(columns={'index'})

##### Agg Stats

In [7]:
all_players_stats = pd.DataFrame()


for match_data in all_matches:
    # Récupérer les statistiques du match
    stats = match_data['agg_stats']
    
    # Joindre les stats des joueurs
    Stats_joueurs = joueurs.merge(stats, on='name', how='left')
    
    # Ajouter les stats agrégées des joueurs à la liste générale
    all_players_stats = pd.concat([all_players_stats, Stats_joueurs])

# Faire la somme des stats par joueur
all_players_stats_grouped = all_players_stats.groupby('name').sum().reset_index()
all_players_stats_grouped['ADR'] = round(all_players_stats_grouped['damage_total'] / all_players_stats_grouped["total_rounds_played"],2)
all_players_stats_grouped['HS %'] = round((all_players_stats_grouped['headshot_kills_total'] / all_players_stats_grouped['kills_total'])*100,2)
all_players_stats_grouped['KPR'] = round((all_players_stats_grouped['kills_total'] / all_players_stats_grouped['total_rounds_played']), 2)
all_players_stats_grouped['K/D'] = round((all_players_stats_grouped['kills_total'] / all_players_stats_grouped['deaths_total']),2)
all_players_stats_grouped = all_players_stats_grouped.rename(columns={"headshot_kills_total":"HS","kills_total":"Kills","assists_total":"Assists","deaths_total":"Deaths","3k_rounds_total":'3K', "4k_rounds_total":'4K', "ace_rounds_total":'5K',"damage_total":"Damages","utility_damage_total":"Utility Damages","enemies_flashed_total":"Flashed Ennemies"})
all_players_stats_grouped = all_players_stats_grouped.drop(columns={"tick"})
nouvel_ordre_colonnes = [
    'name', 'steamid', 'total_rounds_played', 'Kills', 'Deaths', 'Assists', 
    'K/D', 'Damages', 'ADR', 'KPR', 'HS', 'HS %', '5K', '4K', '3K', 'mvps', 'alive_time_total'
]
all_players_stats_grouped = all_players_stats_grouped[nouvel_ordre_colonnes]

In [8]:
all_players_stats_grouped

,name,steamid,total_rounds_played,Kills,Deaths,Assists,K/D,Damages,ADR,KPR,HS,HS %,5K,4K,3K,mvps,alive_time_total
0,-silentGG,153122396222382356,66,35,43,14,0.81,3941,59.71,0.53,18,51.43,0,0,2,4,2994
1,BELDIYA00,153122396753224566,66,66,41,15,1.61,7338,111.18,1.00,42,63.64,0,3,3,11,2682
2,OzzieOzz,153122396133322142,66,48,39,8,1.23,4569,69.23,0.73,25,52.08,0,2,2,7,2748
3,Spiritix,153122396346721750,66,49,43,16,1.14,5264,79.76,0.74,21,42.86,0,0,4,9,2498
4,godofbaldz,153122396519654942,66,47,39,14,1.21,4874,73.85,0.71,7,14.89,0,1,3,4,2597


##### AGG utils

In [186]:
# Initialiser un DataFrame vide pour stocker les stats d'utilitaire des joueurs
all_utils_stats = pd.DataFrame()

# Parcourir chaque match dans 'all_matches'
for match_data in all_matches:
    # Extraction des DataFrames nécessaires
    all_hits_regs = match_data['damage_info']
    stats_utils = match_data['agg_stats']
    trade = match_data['trade_info']
    
    # Calcul des flash assist kills
    flash_assist_kill = trade[trade['assistedflash'] == True].groupby('assister_name').size().reset_index(name='Flash_assist')
    flash_assist_kill = flash_assist_kill.rename(columns={'assister_name':'name'})
    
    # Calcul des dégâts HE
    he_dmg = all_hits_regs[all_hits_regs["weapon"] == "hegrenade"]
    he_dmg = he_dmg.groupby('attacker_name').sum('dmg_health').reset_index()
    he_dmg = he_dmg.rename(columns={'attacker_name':'name'})
    
    # Calcul des dégâts molotov/incendiaire
    molotov_dmg = all_hits_regs[(all_hits_regs["weapon"] == "molotov") | (all_hits_regs["weapon"] == "inferno")]
    molotov_dmg = molotov_dmg.groupby('attacker_name').sum('dmg_health').reset_index()
    molotov_dmg = molotov_dmg.rename(columns={'attacker_name':'name'})
    
    # Fusion des dégâts HE et molotov
    Utils_joueurs = joueurs.merge(he_dmg[['name','dmg_health']], on='name', how='left').fillna(0)
    Utils_joueurs = Utils_joueurs.merge(molotov_dmg[['name','dmg_health']], on='name', how='left').fillna(0)
    Utils_joueurs = Utils_joueurs.rename(columns={'dmg_health_x':'He_dmg','dmg_health_y':'Fire_dmg'})
    Utils_joueurs['Total_utility_dmg'] = Utils_joueurs['He_dmg'] + Utils_joueurs['Fire_dmg']
    
    # Ajouter les flashs et autres stats
    Utils_joueurs = Utils_joueurs.merge(stats_utils[['name','enemies_flashed_total']], on='name', how='left')
    Utils_joueurs = Utils_joueurs.merge(flash_assist_kill, on='name', how='left')
    
    # Remplir les valeurs manquantes et convertir en entier
    Utils_joueurs[['Flash_assist','He_dmg','Fire_dmg','Total_utility_dmg']] = Utils_joueurs[['Flash_assist','He_dmg','Fire_dmg','Total_utility_dmg']].fillna(0).astype(int)
    
    # Concaténer les stats pour ce match avec les autres
    all_utils_stats = pd.concat([all_utils_stats, Utils_joueurs])

# Agréger les résultats pour tous les matchs
all_utils_stats_grouped = all_utils_stats.groupby('name').sum()

# Afficher les résultats finaux
all_utils_stats_grouped.reset_index()


,name,He_dmg,Fire_dmg,Total_utility_dmg,enemies_flashed_total,Flash_assist
0,-silentGG,108,221,329,14,1
1,BELDIYA00,428,157,585,33,0
2,OzzieOzz,273,10,283,16,0
3,Spiritix,214,112,326,9,0
4,godofbaldz,336,89,425,50,4


##### AGG entries

In [187]:
match_data['trade_info']

,assistedflash,assister_name,assister_steamid,attacker_name,attacker_steamid,attackerblind,distance,dmg_armor,dmg_health,dominated,...,weapon_itemid,weapon_originalowner_xuid,wipe,player_died_time,team_clan_name_attacker,side_attacker,name_x,team_clan_name_user,side_user,name_y
0,False,None,None,JustAnotherL,76561198264064168,False,16.071526,0,124,0,...,35034331466,,0,19.328125,KREATURES,CT,JustAnotherL,zobrux,TERRORIST,-silentGG
1,False,JustAnotherL,76561198264064168,MasaIDK_,76561198358546254,False,10.993015,0,129,0,...,13069782900,,0,20.593750,KREATURES,CT,MasaIDK_,zobrux,TERRORIST,Spiritix
2,False,None,None,OzzieOzz,76561198066661071,False,17.510454,0,96,0,...,9625445227,,0,23.015625,zobrux,TERRORIST,OzzieOzz,KREATURES,CT,fykseN
3,False,None,None,BELDIYA00,76561198376612283,False,14.075491,0,101,0,...,0,,0,24.218750,zobrux,TERRORIST,BELDIYA00,KREATURES,CT,JustAnotherL
4,False,None,None,godofbaldz,76561198259827471,False,5.924605,9,17,0,...,17635661541,,0,26.375000,zobrux,TERRORIST,godofbaldz,KREATURES,CT,MasaIDK_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,False,None,None,godofbaldz,76561198259827471,False,29.865206,1,110,0,...,33914766179,,0,14.031250,zobrux,TERRORIST,godofbaldz,KREATURES,CT,MasaIDK_
197,False,None,None,godofbaldz,76561198259827471,False,33.921307,1,136,0,...,33914766179,,0,53.906250,zobrux,TERRORIST,godofbaldz,KREATURES,CT,ADEODATUSS
198,False,None,None,godofbaldz,76561198259827471,False,27.526186,1,137,0,...,33914766179,,0,57.812500,zobrux,TERRORIST,godofbaldz,KREATURES,CT,JustAnotherL
199,False,None,None,OzzieOzz,76561198066661071,False,20.211832,15,108,0,...,35297643185,,0,70.890625,zobrux,TERRORIST,OzzieOzz,KREATURES,CT,fykseN


In [188]:
# Initialiser un DataFrame vide pour stocker les stats d'entry et open
all_opening_stats = pd.DataFrame()

# Parcourir chaque match dans 'all_matches'
for match_data in all_matches:
    trade = match_data['trade_info']
    team_V3 = match_data['team_info']
    
    
    # Identifier le premier kill de chaque round
    first_duel = trade.groupby('total_rounds_played').first().reset_index()

    # Initialiser un DataFrame pour stocker les résultats par joueur pour ce match
    opening_stats = pd.DataFrame({
        'name': trade['attacker_name'].unique(),
        'entry_attempts(T)': 0,
        'entry_successes(T)': 0,
        'open_attempts(CT)': 0,
        'open_successes(CT)': 0
    })

    # Parcourir chaque premier kill pour déterminer si c'est une tentative d'entry (T) ou d'open (CT)
    for _, row in first_duel.iterrows():
        attacker = row['attacker_name']
        user = row['user_name']
        side_user = row['side_user']

        # Si le joueur tué est un CT, c'est une tentative d'entry pour un Terrorist
        if side_user == 'CT':
            opening_stats.loc[opening_stats['name'] == attacker, 'entry_attempts(T)'] += 1
            opening_stats.loc[opening_stats['name'] == attacker, 'entry_successes(T)'] += 1
            opening_stats.loc[opening_stats['name'] == user, 'open_attempts(CT)'] += 1

        # Si le joueur tué est un Terroriste, c'est une tentative d'open pour un CT
        elif side_user == 'TERRORIST':
            opening_stats.loc[opening_stats['name'] == attacker, 'open_attempts(CT)'] += 1
            opening_stats.loc[opening_stats['name'] == attacker, 'open_successes(CT)'] += 1
            opening_stats.loc[opening_stats['name'] == user, 'entry_attempts(T)'] += 1

    # Concaténer les résultats pour ce match avec les autres
    all_opening_stats = pd.concat([all_opening_stats, opening_stats])

# Agréger les résultats pour tous les matchs
all_opening_stats_grouped = all_opening_stats.groupby('name').sum().reset_index()

# Calcul des pourcentages de réussite
all_opening_stats_grouped["%_entry_success(T)"] = round((all_opening_stats_grouped["entry_successes(T)"] / all_opening_stats_grouped["entry_attempts(T)"])*100, 0)
all_opening_stats_grouped["%_open_success(CT)"] = round((all_opening_stats_grouped["open_successes(CT)"] / all_opening_stats_grouped["open_attempts(CT)"])*100, 0)

# Réorganiser les colonnes
column_order = ['name', 'entry_attempts(T)', 'entry_successes(T)', '%_entry_success(T)', 'open_attempts(CT)', 'open_successes(CT)', '%_open_success(CT)']
all_opening_stats_grouped = all_opening_stats_grouped[column_order]

# Fusionner avec le DataFrame 'joueurs'
entry_stats = joueurs.merge(all_opening_stats_grouped, on='name', how='left')

# Afficher les résultats finaux
entry_stats


,name,entry_attempts(T),entry_successes(T),%_entry_success(T),open_attempts(CT),open_successes(CT),%_open_success(CT)
0,-silentGG,6,3,50.0,4,2,50.0
1,OzzieOzz,4,2,50.0,5,1,20.0
2,godofbaldz,9,6,67.0,7,4,57.0
3,BELDIYA00,7,5,71.0,3,3,100.0
4,Spiritix,7,5,71.0,14,11,79.0


##### AGG Trade

In [189]:
# Initialiser des dictionnaires globaux pour stocker les Traded Deaths (T) et Trade Kills (T) cumulés
traded_deaths_global = {}
trade_kills_global = {}

# Parcourir chaque match dans 'all_matches'
for match_data in all_matches:
    trade = match_data['trade_info']
    
    # Initialiser des dictionnaires pour stocker les Traded Deaths (T) et Trade Kills (T) pour ce match
    traded_deaths = {}
    trade_kills = {}

    # Parcourir toutes les lignes du DataFrame pour chercher les morts des terroristes
    for i, row in trade.iterrows():
        if row['side_user'] == 'TERRORIST':
            # Extraire les informations de la ligne courante
            terrorist_name = row['user_name']
            attacker_name = row['attacker_name']
            death_time = row['player_died_time']
            round_number = row['total_rounds_played']

            # Chercher si l'attaquant (CT) est tué dans les 2 secondes après avoir tué le terroriste
            subsequent_kills = trade[(trade['user_name'] == attacker_name) & 
                                     (trade['player_died_time'] > death_time) & 
                                     (trade['player_died_time'] <= death_time + 2)]

            # Si un traded death est trouvé
            if not subsequent_kills.empty:
                # Assurer que la mort de l'attaquant se produit dans le même round que la mort du terroriste
                if all(subsequent_kills['total_rounds_played'] == round_number):
                    if terrorist_name not in traded_deaths:
                        traded_deaths[terrorist_name] = {'count': 0, 'rounds': []}
                    traded_deaths[terrorist_name]['count'] += 1
                    traded_deaths[terrorist_name]['rounds'].append(round_number)

                    # Chercher si le trade kill vient d'un coéquipier du terroriste
                    for _, sub_kill in subsequent_kills.iterrows():
                        trade_killer = sub_kill['attacker_name']
                        if trade_killer != terrorist_name:
                            if trade_killer not in trade_kills:
                                trade_kills[trade_killer] = {'count': 0, 'rounds': []}
                            trade_kills[trade_killer]['count'] += 1
                            trade_kills[trade_killer]['rounds'].append(round_number)

    # Ajouter les résultats de ce match aux dictionnaires globaux
    for player, data in traded_deaths.items():
        if player not in traded_deaths_global:
            traded_deaths_global[player] = 0
        traded_deaths_global[player] += data['count']
    
    for player, data in trade_kills.items():
        if player not in trade_kills_global:
            trade_kills_global[player] = 0
        trade_kills_global[player] += data['count']

# Convertir les dictionnaires globaux en DataFrames pour afficher les résultats cumulés
traded_deaths_list = [(player, count) for player, count in traded_deaths_global.items()]
trade_kills_list = [(player, count) for player, count in trade_kills_global.items()]

traded_deaths_df = pd.DataFrame(traded_deaths_list, columns=['name', 'Traded Deaths (T)'])
trade_kills_df = pd.DataFrame(trade_kills_list, columns=['name', 'Trade Kills (T)'])


In [190]:
trade_stats = joueurs.merge(trade_kills_df, on='name',how='left')
trade_stats['Trade Kills (T)'] = trade_stats['Trade Kills (T)'].fillna(0).astype(int)
trade_stats = trade_stats.merge(traded_deaths_df, on='name',how='left')
trade_stats['Traded Deaths (T)'] = trade_stats['Traded Deaths (T)'].fillna(0).astype(int)
trade_stats

,name,Trade Kills (T),Traded Deaths (T)
0,-silentGG,4,3
1,OzzieOzz,2,3
2,godofbaldz,1,2
3,BELDIYA00,3,1
4,Spiritix,0,1


##### AGG ecofrags

In [194]:
# Initialiser un DataFrame global pour stocker les kills par catégorie cumulés
all_team_eco_kills = pd.DataFrame()

# Parcourir chaque match dans 'all_matches'
for match_data in all_matches:
    trade_df = match_data['trade_info']  # DataFrame des kills
    eco_by_team = match_data['eco_info']  # DataFrame de l'économie de l'équipe
    game_id = match_data['team_info']['Game_id'].iloc[0]  # Utiliser le même Game_id pour cette partie

    # Ajouter la colonne Game_id aux kills et à l'économie pour différencier les parties
    trade_df['Game_id'] = game_id
    eco_by_team['Game_id'] = game_id

    # Fusionner les données de kills avec l'économie totale de l'équipe de l'attaquant
    df_merged_team = pd.merge(trade_df, eco_by_team, how='left', 
                              left_on=['total_rounds_played', 'team_clan_name_user', 'Game_id'], 
                              right_on=['total_rounds_played', 'team_clan_name', 'Game_id'])

    # Appliquer la fonction de catégorisation pour classer les kills
    def categorize_kill(row):
        if row['total_rounds_played'] in [1, 13]:
            return 'Pistol round'
        elif row['current_equip_value'] <= 3500:
            return 'Anti eco kill'
        elif row['current_equip_value'] <= 18000:
            return 'Anti force buy kill'
        else:
            return 'Full buy kill'

    # Appliquer la fonction sur les kills du match
    df_merged_team['kill_category'] = df_merged_team.apply(categorize_kill, axis=1)

    # Utiliser pivot_table pour compter les kills par catégorie et par joueur
    team_eco_kills = df_merged_team.pivot_table(index='attacker_name', columns='kill_category', aggfunc='size', fill_value=0)

    # Ajouter les résultats pour ce match aux résultats cumulés
    all_team_eco_kills = pd.concat([all_team_eco_kills, team_eco_kills])

# Agréger les résultats cumulés pour tous les matchs
all_team_eco_kills_grouped = all_team_eco_kills.groupby('attacker_name').sum()

# Afficher les résultats finaux
all_team_eco_kills_grouped.head()

kill_category,Anti eco kill,Anti force buy kill,Full buy kill,Pistol round
attacker_name,,,,
-Weppling,0,5,14,1
-silentGG,2,2,29,2
3PARABELLUM,3,2,9,5
ADEODATUSS,1,2,19,2
BELDIYA00,5,6,52,3


In [193]:
eco_stats = pd.merge(joueurs,all_team_eco_kills_grouped,left_on='name',right_on='attacker_name',how='left')
eco_stats

,name,Anti eco kill,Anti force buy kill,Full buy kill,Pistol round
0,-silentGG,2,2,29,2
1,OzzieOzz,3,5,36,4
2,godofbaldz,5,3,36,3
3,BELDIYA00,5,6,52,3
4,Spiritix,5,12,30,2


#### Analyse Saison

- Nombre de victoires
- Nombre de défaites
- plus long winstreak
- Winrate par cartes
- % pistols remportés
- largest win
- closest game
- MVP de l'équipe

In [16]:
all_matches[0]['team_info']

,side,team_clan_name,tick,steamid,name,Game_id
60780,CT,zobrux,6084,76561198259827471,godofbaldz,VS_Eldsjalar
60781,CT,zobrux,6084,76561198376612283,BELDIYA00,VS_Eldsjalar
60787,CT,zobrux,6084,76561198173360875,Spiritix,VS_Eldsjalar
60789,CT,zobrux,6084,76561198066661071,OzzieOzz,VS_Eldsjalar
60790,CT,zobrux,6087,76561198259827471,godofbaldz,VS_Eldsjalar
...,...,...,...,...,...,...
2850020,TERRORIST,zobrux,285041,76561198376612283,BELDIYA00,VS_Eldsjalar
2850022,TERRORIST,zobrux,285041,76561198259827471,godofbaldz,VS_Eldsjalar
2850024,TERRORIST,zobrux,285041,76561198111191178,-silentGG,VS_Eldsjalar
2850026,TERRORIST,zobrux,285041,76561198173360875,Spiritix,VS_Eldsjalar


In [8]:
all_matches[0]['round_info']

,reason,tick,total_rounds_played,winner
1,ct_killed,12040,1,T
2,ct_killed,18764,2,T
3,ct_killed,25579,3,T
4,bomb_defused,34960,4,CT
5,t_killed,41945,5,CT
6,t_killed,45588,6,CT
7,bomb_defused,52792,7,CT
8,t_killed,57490,8,CT
9,unknown,67316,9,T
10,ct_killed,76940,10,T


In [ ]:
mathis_suce_moi = all_matches[0]['round_info'].merge(all_matches[0]['team_info'], on='')